<a href="http://colab.research.google.com/github/dipanjanS/nlp_workshop_odsc19/blob/master/Module05%20-%20NLP%20Applications/Project07D%20-%20Text%20Classification%20Deep%20Transfer%20Learning%20with%20Universal%20Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis - Text Classification with Universal Embeddings

Textual data in spite of being highly unstructured, can be classified into two major types of documents. 
- __Factual documents__ which typically depict some form of statements or facts with no specific feelings or emotion attached to them. These are also known as objective documents. 
- __Subjective documents__ on the other hand have text which expresses feelings, mood, emotions and opinion. 

Sentiment Analysis is also popularly known as opinion analysis or opinion mining. The key idea is to use techniques from text analytics, NLP, machine learning and linguistics to extract important information or data points from unstructured text. This in turn can help us derive the sentiment from text data

![](https://github.com/dipanjanS/data_science_for_all/blob/master/tds_deep_transfer_learning_nlp_classification/sentiment_cover.png?raw=1)

Here we will be looking at building supervised sentiment analysis classification models thanks to the advantage of labeled data! The dataset we will be working with is the IMDB Large Movie Review Dataset having 50000 reviews classified into positive and negative sentiment. I have provided a compressed version of the dataset in this repository itself for your benefit!

Do remember that the focus here is not sentiment analysis but text classification by leveraging universal sentence embeddings.

![](https://github.com/dipanjanS/data_science_for_all/blob/master/tds_deep_transfer_learning_nlp_classification/sample_classification.png?raw=1)

We will leverage the following sentence encoders here for demonstration from [TensorFlow Hub](https://tfhub.dev/):

- [__Neural-Net Language Model (nnlm-en-dim128)__](https://tfhub.dev/google/nnlm-en-dim128/1)
- [__Universal Sentence Encoder (universal-sentence-encoder)__](https://tfhub.dev/google/universal-sentence-encoder/2)


_Developed by [Dipanjan (DJ) Sarkar](https://www.linkedin.com/in/dipanzan/)_

# Install Dependencies

In [2]:
!pip install tensorflow-hub
!pip install textsearch
!pip install contractions
!pip install tqdm
import nltk
nltk.download('punkt')

     |████████████████████████████████| 321 kB 5.4 MB/s 
     |████████████████████████████████| 284 kB 40.6 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85450 sha256=c7aa7fa6b7b6444ed25317e1af0f3ff38b4cddf85d4b3f2bbe0b8b1dc951a8a0
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Load up Dependencies

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd

# Check if GPU is available for use!

In [4]:
!nvidia-smi

Fri Dec 31 06:11:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [6]:
tf.test.gpu_device_name()

'/device:GPU:0'

# Load and View Dataset

In [7]:
dataset = pd.read_csv('https://github.com/dipanjanS/nlp_workshop_dhs18/raw/master/Unit%2011%20-%20Sentiment%20Analysis%20-%20Unsupervised%20Learning/movie_reviews.csv.bz2', compression='bz2')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [8]:
dataset['sentiment'] = [1 if sentiment == 'positive' else 0 for sentiment in dataset['sentiment'].values]
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


# Build train, validation and test datasets

In [9]:
reviews = dataset['review'].values
sentiments = dataset['sentiment'].values

train_reviews = reviews[:31500]
train_sentiments = sentiments[:31500]

val_reviews = reviews[31500:35000]
val_sentiments = sentiments[31500:35000]

test_reviews = reviews[35000:]
test_sentiments = sentiments[35000:]
train_reviews.shape, val_reviews.shape, test_reviews.shape

((31500,), (3500,), (15000,))

# Basic Text Wrangling

In [10]:
import contractions
from bs4 import BeautifulSoup
import numpy as np
import re
import tqdm
import unicodedata


def strip_html_tags(text):
  soup = BeautifulSoup(text, "html.parser")
  [s.extract() for s in soup(['iframe', 'script'])]
  stripped_text = soup.get_text()
  stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
  return stripped_text

def remove_accented_chars(text):
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  return text

def pre_process_corpus(docs):
  norm_docs = []
  for doc in tqdm.tqdm(docs):
    doc = strip_html_tags(doc)
    doc = doc.translate(doc.maketrans("\n\t\r", "   "))
    doc = doc.lower()
    doc = remove_accented_chars(doc)
    doc = contractions.fix(doc)
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = re.sub(' +', ' ', doc)
    doc = doc.strip()  
    norm_docs.append(doc)
  
  return norm_docs

In [11]:
train_reviews = np.array(pre_process_corpus(train_reviews))
val_reviews = np.array(pre_process_corpus(val_reviews))
test_reviews = np.array(pre_process_corpus(test_reviews))

100%|██████████| 15000/15000 [00:08<00:00, 1789.80it/s]


# Build Data Ingestion Functions

In [12]:
# Training input on the whole training set with no limit on training epochs.
train_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    {'sentence': train_reviews}, train_sentiments, 
    batch_size=256, num_epochs=None, shuffle=True)

In [13]:
# Prediction on the whole training set.
predict_train_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    {'sentence': train_reviews}, train_sentiments, shuffle=False)

In [14]:
# Prediction on the whole validation set.
predict_val_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    {'sentence': val_reviews}, val_sentiments, shuffle=False)

In [15]:
# Prediction on the test set.
predict_test_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    {'sentence': test_reviews}, test_sentiments, shuffle=False)

# Build Deep Learning Model with Universal Sentence Encoder

In [16]:
embedding_feature = hub.text_embedding_column(
    key='sentence', 
    module_spec="https://tfhub.dev/google/universal-sentence-encoder/2",
    trainable=False)

In [17]:
dnn = tf.compat.v1.estimator.DNNClassifier(
          hidden_units=[512, 128],
          feature_columns=[embedding_feature],
          n_classes=2,
          activation_fn=tf.compat.v1.nn.relu,
          dropout=0.1,
          optimizer=tf.compat.v1.train.AdagradOptimizer(learning_rate=0.005))

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpnnjunsck', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpnnjunsck', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


### Train for approx 10 epochs

In [18]:
256*1200 / 31500

9.752380952380953

# Model Training

In [19]:
# tf.logging.set_verbosity(tf.logging.ERROR)
import time

TOTAL_STEPS = 1200
STEP_SIZE = 300
for step in range(0, TOTAL_STEPS, STEP_SIZE):
    print()
    print('-'*100)
    print('Training for step:', step, '-', step+STEP_SIZE)
    start_time = time.time()
    dnn.train(input_fn=train_input_fn, steps=STEP_SIZE)
    elapsed_time = time.time() - start_time
    print('Train Time (s):', elapsed_time)
    print('Eval Metrics (Train):', dnn.evaluate(input_fn=predict_train_input_fn))
    print('Eval Metrics (Validation):', dnn.evaluate(input_fn=predict_val_input_fn))


----------------------------------------------------------------------------------------------------
Training for step: 0 - 300
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
To construct input pipelines, use the `tf.data` module.


Instructions for updating:
To construct input pipelines, use the `tf.data` module.


Instructions for updating:
To construct input pipelines, use the `tf.data` module.


...

INFO:tensorflow:Saving dict for global step 1200: accuracy = 0.86628574, accuracy_baseline = 0.50514287, auc = 0.93882114, auc_precision_recall = 0.9350327, average_loss = 0.31668988, global_step = 1200, label/mean = 0.49485713, loss = 39.586235, precision = 0.85307264, prediction/mean = 0.5098917, recall = 0.8816397


INFO:tensorflow:Saving dict for global step 1200: accuracy = 0.86628574, accuracy_baseline = 0.50514287, auc = 0.93882114, auc_precision_recall = 0.9350327, average_loss = 0.31668988, global_step = 1200, label/mean = 0.49485713, loss = 39.586235, precision = 0.85307264, prediction/mean = 0.5098917, recall = 0.8816397


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1200: /tmp/tmpnnjunsck/model.ckpt-1200


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1200: /tmp/tmpnnjunsck/model.ckpt-1200


Eval Metrics (Validation): {'accuracy': 0.86628574, 'accuracy_baseline': 0.50514287, 'auc': 0.93882114, 'auc_precision_recall': 0.9350327, 'average_loss': 0.31668988, 'label/mean': 0.49485713, 'loss': 39.586235, 'precision': 0.85307264, 'prediction/mean': 0.5098917, 'recall': 0.8816397, 'global_step': 1200}


# Model Evaluation

In [20]:
dnn.evaluate(input_fn=predict_train_input_fn)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving dict for global step 1200: accuracy = 0.8702857, accuracy_baseline = 0.500254, auc = 0.94439405, auc_precision_recall = 0.9437617, average_loss = 0.30198246, global_step = 1200, label/mean = 0.500254, loss = 38.511932, precision = 0.86964786, prediction/mean = 0.5022442, recall = 0.87130344


INFO:tensorflow:Saving dict for global step 1200: accuracy = 0.8702857, accuracy_baseline = 0.500254, auc = 0.94439405, auc_precision_recall = 0.9437617, average_loss = 0.30198246, global_step = 1200, label/mean = 0.500254, loss = 38.511932, precision = 0.86964786, prediction/mean = 0.5022442, recall = 0.87130344


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1200: /tmp/tmpnnjunsck/model.ckpt-1200


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1200: /tmp/tmpnnjunsck/model.ckpt-1200


{'accuracy': 0.8702857,
 'accuracy_baseline': 0.500254,
 'auc': 0.94439405,
 'auc_precision_recall': 0.9437617,
 'average_loss': 0.30198246,
 'global_step': 1200,
 'label/mean': 0.500254,
 'loss': 38.511932,
 'precision': 0.86964786,
 'prediction/mean': 0.5022442,
 'recall': 0.87130344}

In [21]:
dnn.evaluate(input_fn=predict_test_input_fn)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving dict for global step 1200: accuracy = 0.8628, accuracy_baseline = 0.5006667, auc = 0.9374881, auc_precision_recall = 0.9365008, average_loss = 0.31961274, global_step = 1200, label/mean = 0.5006667, loss = 40.62874, precision = 0.86366063, prediction/mean = 0.50349826, recall = 0.8620506


INFO:tensorflow:Saving dict for global step 1200: accuracy = 0.8628, accuracy_baseline = 0.5006667, auc = 0.9374881, auc_precision_recall = 0.9365008, average_loss = 0.31961274, global_step = 1200, label/mean = 0.5006667, loss = 40.62874, precision = 0.86366063, prediction/mean = 0.50349826, recall = 0.8620506


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1200: /tmp/tmpnnjunsck/model.ckpt-1200


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1200: /tmp/tmpnnjunsck/model.ckpt-1200


{'accuracy': 0.8628,
 'accuracy_baseline': 0.5006667,
 'auc': 0.9374881,
 'auc_precision_recall': 0.9365008,
 'average_loss': 0.31961274,
 'global_step': 1200,
 'label/mean': 0.5006667,
 'loss': 40.62874,
 'precision': 0.86366063,
 'prediction/mean': 0.50349826,
 'recall': 0.8620506}

# Build a Generic Model Trainer on any Input Sentence Encoder

In [26]:
import time

TOTAL_STEPS = 1200
STEP_SIZE = 400

my_checkpointing_config = tf.compat.v1.estimator.RunConfig(
    keep_checkpoint_max = 1,       # Retain the 2 most recent checkpoints.
)

def train_and_evaluate_with_sentence_encoder(hub_module, train_module=False, path=''):
    embedding_feature = hub.text_embedding_column(
        key='sentence', module_spec=hub_module, trainable=train_module)
  
    print()
    print('='*100)
    print('Training with', hub_module)
    print('Trainable is:', train_module)
    print('='*100)
  
    dnn = tf.compat.v1.estimator.DNNClassifier(
            hidden_units=[512, 128],
            feature_columns=[embedding_feature],
            n_classes=2,
            activation_fn=tf.nn.relu,
            dropout=0.1,
            optimizer=tf.compat.v1.train.AdagradOptimizer(learning_rate=0.005),
            model_dir=path,
            config=my_checkpointing_config)

    for step in range(0, TOTAL_STEPS, STEP_SIZE):
        print('-'*100)
        print('Training for step:', step, '-', step+STEP_SIZE)
        start_time = time.time()
        dnn.train(input_fn=train_input_fn, steps=STEP_SIZE)
        elapsed_time = time.time() - start_time
        print('Train Time (s):', elapsed_time)
        print('Eval Metrics (Train):', dnn.evaluate(input_fn=predict_train_input_fn))
        print('Eval Metrics (Validation):', dnn.evaluate(input_fn=predict_val_input_fn))

    train_eval_result = dnn.evaluate(input_fn=predict_train_input_fn)
    test_eval_result = dnn.evaluate(input_fn=predict_test_input_fn)

    return {
      "Model Dir": dnn.model_dir,
      "Training Accuracy": train_eval_result["accuracy"],
      "Test Accuracy": test_eval_result["accuracy"],
      "Training AUC": train_eval_result["auc"],
      "Test AUC": test_eval_result["auc"],
      "Training Precision": train_eval_result["precision"],
      "Test Precision": test_eval_result["precision"],
      "Training Recall": train_eval_result["recall"],
      "Test Recall": test_eval_result["recall"]
    }

# Train Deep Learning Models on difference Sentence Encoders
- NNLM - pre-trained and fine-tuning
- USE - pre-trained and fine-tuning

In [27]:
# tf.logging.set_verbosity(tf.logging.ERROR)

results = {}

results["nnlm-en-dim128"] = train_and_evaluate_with_sentence_encoder(
    "https://tfhub.dev/google/nnlm-en-dim128/1", path='/storage/models/nnlm-en-dim128_fr/')

results["nnlm-en-dim128-with-training"] = train_and_evaluate_with_sentence_encoder(
    "https://tfhub.dev/google/nnlm-en-dim128/1", train_module=True, path='/storage/models/nnlm-en-dim128_ft/')

results["use-512"] = train_and_evaluate_with_sentence_encoder(
    "https://tfhub.dev/google/universal-sentence-encoder/2", path='/storage/models/use-512_fr/')

results["use-512-with-training"] = train_and_evaluate_with_sentence_encoder(
    "https://tfhub.dev/google/universal-sentence-encoder/2", train_module=True, path='/storage/models/use-512_ft/')


Training with https://tfhub.dev/google/nnlm-en-dim128/1
Trainable is: False
INFO:tensorflow:Using config: {'_model_dir': '/storage/models/nnlm-en-dim128_fr/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/storage/models/nnlm-en-dim128_fr/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


----------------------------------------------------------------------------------------------------
Training for step: 0 - 400
INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving dict for global step 1200: accuracy = 0.82913333, accuracy_baseline = 0.5006667, auc = 0.90967304, auc_precision_recall = 0.91065013, average_loss = 0.3812078, global_step = 1200, label/mean = 0.5006667, loss = 48.458614, precision = 0.83538985, prediction/mean = 0.500697, recall = 0.8203728


INFO:tensorflow:Saving dict for global step 1200: accuracy = 0.82913333, accuracy_baseline = 0.5006667, auc = 0.90967304, auc_precision_recall = 0.91065013, average_loss = 0.3812078, global_step = 1200, label/mean = 0.5006667, loss = 48.458614, precision = 0.83538985, prediction/mean = 0.500697, recall = 0.8203728


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1200: /storage/models/nnlm-en-dim128_fr/model.ckpt-1200


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1200: /storage/models/nnlm-en-dim128_fr/model.ckpt-1200


NameError: ignored

# Model Evaluations

In [ ]:
results_df = pd.DataFrame.from_dict(results, orient="index")
results_df

In [ ]:
best_model_dir = results_df[results_df['Test Accuracy'] == results_df['Test Accuracy'].max()]['Model Dir'].values[0]
best_model_dir

In [ ]:
embedding_feature = hub.text_embedding_column(
        key='sentence', module_spec="https://tfhub.dev/google/universal-sentence-encoder/2", trainable=True)

dnn = tf.compat.v1.estimator.DNNClassifier(
            hidden_units=[512, 128],
            feature_columns=[embedding_feature],
            n_classes=2,
            activation_fn=tf.nn.relu,
            dropout=0.1,
            optimizer=tf.compat.v1.train.AdagradOptimizer(learning_rate=0.005),
            model_dir=best_model_dir)
dnn

In [ ]:
def get_predictions(estimator, input_fn):
    return [x["class_ids"][0] for x in estimator.predict(input_fn=input_fn)]

In [ ]:
predictions = get_predictions(estimator=dnn, input_fn=predict_test_input_fn)
predictions[:10]

In [ ]:
!pip install seaborn

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

with tf.Session() as session:
    cm = tf.confusion_matrix(test_sentiments, predictions).eval()

LABELS = ['negative', 'positive']
sns.heatmap(cm, annot=True, xticklabels=LABELS, yticklabels=LABELS, fmt='g')
xl = plt.xlabel("Predicted")
yl = plt.ylabel("Actuals")

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_true=test_sentiments, y_pred=predictions, target_names=LABELS))